In [1]:
import spacy
from textacy import extract

spacy.info()


    Info about spaCy

    spaCy version      2.0.16         
    Location           /home/abevieiramota/anaconda3/envs/webnlg/lib/python3.6/site-packages/spacy
    Platform           Linux-4.4.0-43-Microsoft-x86_64-with-debian-stretch-sid
    Python version     3.6.7          
    Models             en, en_core_web_lg



{'spaCy version': '2.0.16',
 'Location': '/home/abevieiramota/anaconda3/envs/webnlg/lib/python3.6/site-packages/spacy',
 'Platform': 'Linux-4.4.0-43-Microsoft-x86_64-with-debian-stretch-sid',
 'Python version': '3.6.7',
 'Models': 'en, en_core_web_lg'}

In [2]:
nlp = spacy.load('en_core_web_lg')

In [3]:
import os
os.sys.path.insert(0, '../script')
from webnlg import *
from evaluation import *
from lexicalization import *
from sentence_generation import *
from content_selection import SelectAllContentSelection
from collections import defaultdict, Counter
from itertools import groupby, islice
from template_extraction import *
from discourse_structuring import *
from sentence_aggregation import *
from text_generation import TemplateBasedTextGenerator
import networkx.algorithms.isomorphism as iso
from data_alignment import *
import re
from tqdm import tqdm

from textacy import similarity
%matplotlib inline

In [29]:
from importlib import reload
import webnlg
reload(webnlg)
from webnlg import *

In [30]:
v_12 = WebNLGCorpus.load(['dev_1.2', 'train_1.2'])

In [33]:
EDGE_MATCH = iso.categorical_edge_match(['predicate', 's_label', 'o_label'], [None, None, None])

def get_templates_counter(graph):
    
    n_triplas = len(graph.edges())
    
    for g, templates_counter in template_db[n_triplas].items():
        
        if nx.is_isomorphic(graph, g, edge_match=EDGE_MATCH):
            
            return g, templates_counter
    return None, None


replace_agent_bridge_patient_slot = re.compile(r'((AGENT-\d)|(BRIDGE-\d)|(PATIENT-\d))')

template_db = {x: defaultdict(Counter) for x in range(1, 8)}

for e in tqdm(v_12):
    
    delex_graph = e.get_graph(delexicalized=True)
    if delex_graph:
    
        iso_g, _ = get_templates_counter(e.get_graph())

        if iso_g is None:
            iso_g = delex_graph

        n_triplas = len(e.get_data())

        for lexe in e.entry['lexes']:

            template = Template(replace_agent_bridge_patient_slot.sub('{\g<1>}', lexe['template']))

            template_db[n_triplas][iso_g][template] += 1

100%|██████████| 7812/7812 [02:04<00:00, 62.51it/s] 


In [34]:
# content selection
cs = SelectAllContentSelection()

# sentence generation
t = ManualTemplateExtract()
v_12_ntriples_1 = v_12.subset(ntriples=1)
template_db_ = t.extract(v_12_ntriples_1)

mfe = MostFrequentTemplateSentenceGenerator()
mfe.fit(template_db_)

npsg = NearestPredicateTemplateSentenceGenerator(sentence_generator=mfe,
                                                 similarity_metric=similarity.jaro_winkler,
                                                 threshold=0.8)
npsg.fit(template_db_)

fsg = FallBackPipelineSentenceGenerator([mfe, npsg, JustJoinTripleSentenceGenerator()])


# lexicalization
la = LexicalizeAsAligned()
la.fit(v_12_ntriples_1)

# discourse structuring
ds = DoesntSortDiscourseStructuring()

# sentence aggregation
sa = OneSentenceAggregator()

nlg = TemplateBasedTextGenerator(cs, fsg, la, ds, sa)

In [35]:
test = WebNLGCorpus.load("test_with_lex")

In [36]:
def verbalize(graph):
    
    g_iso, templates_counter = get_templates_counter(graph)
    
    if not g_iso:
        
        return None
    
    #TODO: improve get_templates para já retornar o mapeamento
    GM = nx.algorithms.isomorphism.DiGraphMatcher(g_iso, graph)
    
    mapping = next(GM.isomorphisms_iter())
    
    mapping_ = []
    
    for k, v in mapping.items():
        
        if v in la.subject_lexicalization:
            mapping_.append((k, la.subject_lexicalization[v].most_common(1)[0][0]))
        elif v in la.object_lexicalization:
            mapping_.append((k, la.object_lexicalization[v].most_common(1)[0][0]))
        else:
            mapping_.append((k, preprocess_so(v)))
            
    mapping = dict(mapping_)
    
    texts = []
    
    for template, n in templates_counter.most_common():
        
        texts.append((template.fill(mapping), n))
        
    return texts

# https://stackoverflow.com/a/30134039/3662965
def partition(collection):
    if len(collection) == 1:
        yield [ collection ]
        return

    first = collection[0]
    for smaller in partition(collection[1:]):
        # insert `first` in each of the subpartition's subsets
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
        # put `first` in its own subset 
        yield [ [ first ] ] + smaller


def templates(s):
    
    result = []
    
    last_i = 0
    
    data = s.get_data()
    
    while len(data) > 0:
        
        found = False
        
        for i in range(len(data), last_i, -1):
            
            g = get_graph_from_triples(data[:i])
            
            ts = verbalize(g)
            
            if ts:
                
                result.append(ts[0][0])
                data = data[i:]
                last_i = i
                found = True
                break
        if not found:
            return result, data
                
    return result, None

def templates2(s):
    
    for p in partition(s.get_data()):
        
        texts = []
        
        shit = False
        
        for p_ in p:
            
            g = get_graph_from_triples(p_)
            
            ts = verbalize(g)
            
            if not ts:
                
                shit = True
                break
            else:
                texts.append(ts[0][0])
                
        if not shit:
            return texts, None
        
    return None, s.get_data()


def generate(e):
    
    result, data = templates2(e)
    
    texts = []
    if result:

        texts.append('.'.join(result))

    if data:
        texts.append(nlg.predict_entry(data))
        
    return ' '.join(texts)

In [37]:
generated = [] 

for e in test:

    generated.append(generate(e))

In [38]:
import codecs

with codecs.open('../data/models/elmodel3.txt', 'w', 'utf-8') as f:
    
    for text in generated:
        
        f.write("{}\n".format(text))

In [39]:
!python ../evaluation/webnlg2017/webnlg-automatic-evaluation-v2/evaluation_v2.py --team_name elmodel3 --team_filepath ../data/models/elmodel3.txt --outdir ../tmp/elmodel3 

Files creating finished for:  elmodel3


In [40]:
bleu_all_cat = "../tmp/elmodel3/elmodel3_all-cat.txt"

evaluate_texts(bleu_all_cat)

{'bleu': 40.95, 'meteor': 0.3844564946198066, 'ter': 0.5489830867214401}

In [41]:
bleu_all_cat = "../tmp/elmodel/elmodel_all-cat.txt"

evaluate_texts(bleu_all_cat)

{'bleu': 42.27, 'meteor': 0.38013555867054244, 'ter': 0.532704777664466}

In [31]:
evaluate_model(nlg, 'ola')

{'bleu': 44.01, 'meteor': 0.3909775015368552, 'ter': 0.5066825729733884}

In [32]:
template_db['creator']

Counter({{subject} was created by {object} .: 58,
         The creator of {subject} is {object} .: 13,
         {subject} , was created by {object} .: 3,
         {object} created {subject} .: 8,
         {object} is one of the creators of {subject} .: 1,
         {object} is the creator of {subject} .: 13,
         {subject} were created by {object} .: 1,
         {subject} is created by {object} .: 2,
         {subject} is a fictional character created by {object} .: 1,
         {object} was the creator of {subject} .: 3,
         {object} was one of the creators of {subject} .: 1,
         {subject} was created by {object} in the comics genre .: 1,
         {subject} is a comic character created by {object} .: 1,
         {subject} , The Arrow , was created by {object} .: 1,
         {subject} was created at {object} .: 1,
         {subject} was created in {object} .: 1})

# Examples

In [4]:
test = WebNLGCorpus.load(['test_with_lex'])

## Example 1

In [28]:
sample = test.sample(ntriples=3)

text = nlg.predict_entry(sample.get_data())

sample

Triple info: category=Building eid=Id471

	Modified triples:

Alan_B._Miller_Hall | buildingStartDate | "30 March 2007"
Mason_School_of_Business | country | United_States
Alan_B._Miller_Hall | currentTenants | Mason_School_of_Business


	Lexicalizations:

Alan B. Miller Hall was started on March 30,2007 and has The Mason School of Business in the U.S. as a tenant. || 
Alan B. Miller Hall's building opened in 30th March 2007. The Mason School of Business in the United States are the current tenants of Alan B Miller Hall. || 

In [29]:
text

'The construction of alan b miller hall began in 30th march 2007 . the mason school of business is in the united states . the mason school of business are the current tenants of alan b miller hall .'

## Example 2

In [30]:
sample = test.sample()

text = nlg.predict_entry(sample.get_data())

sample

Triple info: category=SportsTeam eid=Id125

	Modified triples:

Michele_Marcolini | club | Atalanta_B.C.


	Lexicalizations:

Michele Marcolini has played for Atalanta BC. || 
Michele Marcolini is part of the Atalanta B.C. club. || 
Michele Marcolini plays for Atalanta B.C. || 

In [31]:
text

'michele marcolini plays for Atalanta B.C. .'